## Defining helpers

In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import pandas as pd

load_dotenv()

engine = create_engine(
    "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}".format(
        dialect="postgresql",
        driver="psycopg2",
        username=os.environ["POSTGRES_USER"],
        password=os.environ["POSTGRES_PASSWORD"],
        host="localhost",
        port=5432,
        database="feedback"
    )
)

def read_sql_query(query):
    with engine.connect() as conn:
        return pd.read_sql_query(sql=text(query), con=conn)
        
def just_execute(query):
    with engine.connect() as conn:
        conn.execute(text(query))
        conn.commit()
    return "DONE"

def explain_analyze(query):
    with engine.connect() as conn:
        return conn.execute(text(f"EXPLAIN ANALYZE {query}")).fetchall()

## Exploring the data

In [2]:
read_sql_query(
    """
    SELECT 
        ut.relname AS table_name,
        pg_size_pretty(pg_relation_size(ut.relid)) AS table_size,
        pg_size_pretty(pg_relation_size(c.reltoastrelid)) AS toast_size,
        pg_size_pretty(pg_total_relation_size(ut.relid) - pg_relation_size(ut.relid)) AS index_size
    FROM pg_catalog.pg_statio_user_tables ut LEFT JOIN pg_class c ON ut.relid = c.oid
    ORDER BY pg_total_relation_size(ut.relid) DESC;
   """
)

,table_name,table_size,toast_size,index_size
0,feedback,17 MB,2896 kB,4784 kB
1,customer,1968 kB,0 bytes,496 kB
2,business,1128 kB,0 bytes,280 kB


In [3]:
read_sql_query("SELECT * FROM business LIMIT 5")

,business_id,name,description,lat,lon,active,registered
0,1,Chan-Clark,Distributed static budgetary management,41.27621,-72.86843,False,2025-01-17 06:47:37+00:00
1,2,"Edwards, Smith and Brown",Business-focused solution-oriented emulation,33.54428,-84.23381,True,2023-02-15 14:17:38+00:00
2,3,Jenkins Group,Digitized intermediate challenge,45.53929,-122.38731,True,2022-12-31 09:18:52+00:00
3,4,Harrington and Sons,Distributed background projection,40.60538,-73.75513,False,2021-06-26 15:39:55+00:00
4,5,Haley LLC,Enhanced 24/7 functionalities,40.65538,-74.38987,True,2024-12-26 03:25:38+00:00


In [4]:
read_sql_query("SELECT * FROM customer LIMIT 5")

,customer_id,name,banned_from,registered
0,1,Mr. Sean Hubbard 196,"[4853, 2, 1079]",2020-11-04 03:07:21+00:00
1,2,Mx. Denise Cameron 722,"[2503, 2536, 5399]",2020-02-23 13:48:23+00:00
2,3,Mx. Nicole Smith 434,"[5389, 8338, 125]",2021-10-28 14:08:36+00:00
3,4,Mrs. Jasmine Simon 982,"[8314, 5138, 1050]",2020-05-29 19:16:19+00:00
4,5,Mr. Judith Burns 377,"[7341, 2339, 6344]",2020-02-10 16:29:22+00:00


In [5]:
read_sql_query("SELECT * FROM feedback LIMIT 5")

,feedback_id,business_id,customer_id,message,last_edited
0,1,1,18428,Lose situation develop let few increase staff....,2026-07-17 17:02:53+00:00
1,2,1,14113,Media space rest theory second fear. Develop m...,2022-07-14 19:42:50+00:00
2,3,1,10276,Talk hot the company strategy that Mrs. Import...,2024-06-18 21:55:34+00:00
3,4,1,15392,It season model environmental. My program itse...,2025-11-06 16:30:11+00:00
4,5,1,882,Book yet field find agreement their analysis. ...,2023-11-21 00:08:00+00:00


### Ensure ONLY indices for primary keys exist

In [6]:
read_sql_query(
    """
    SELECT
      schemaname,
      relname AS table_name,
      indexrelname AS index_name
    FROM pg_stat_user_indexes
    """
)

,schemaname,table_name,index_name
0,public,business,business_pkey
1,public,customer,customer_pkey
2,public,feedback,feedback_pkey


In [7]:
just_execute(
    """
    DROP INDEX IF EXISTS business_name_idx;
    DROP INDEX IF EXISTS business_location_idx;
    DROP INDEX IF EXISTS customer_name_idx;
    DROP INDEX IF EXISTS customer_banned_from_idx;
    DROP INDEX IF EXISTS feedback_business_idx;
    DROP INDEX IF EXISTS feedback_customer_idx;
    DROP INDEX IF EXISTS feedback_message_idx;
    """
)

'DONE'

In [8]:
read_sql_query(
    """
    SELECT
      schemaname,
      relname AS table_name,
      indexrelname AS index_name
    FROM pg_stat_user_indexes
    """
)

,schemaname,table_name,index_name
0,public,business,business_pkey
1,public,customer,customer_pkey
2,public,feedback,feedback_pkey


## Querying 'business' table

### Find by name

In [9]:
explain_analyze(
    """
    SELECT * FROM business WHERE name = 'Edwards, Smith and Brown'
    """
)

[('Seq Scan on business  (cost=0.00..266.00 rows=1 width=80) (actual time=0.007..0.657 rows=1 loops=1)',),
 ("  Filter: (name = 'Edwards, Smith and Brown'::text)",),
 ('  Rows Removed by Filter: 9999',),
 ('Planning Time: 0.122 ms',),
 ('Execution Time: 0.669 ms',)]

In [10]:
just_execute(
    """
    CREATE INDEX IF NOT EXISTS business_name_idx ON business USING BTREE (name);
    ANALYZE business;
    """
)

'DONE'

In [11]:
explain_analyze(
    """
    SELECT * FROM business WHERE name = 'Edwards, Smith and Brown'
    """
)

[('Index Scan using business_name_idx on business  (cost=0.29..8.30 rows=1 width=80) (actual time=0.064..0.065 rows=1 loops=1)',),
 ("  Index Cond: (name = 'Edwards, Smith and Brown'::text)",),
 ('Planning Time: 0.220 ms',),
 ('Execution Time: 0.078 ms',)]

### Find nearest

In [12]:
explain_analyze(
    """
    SELECT *, earth_distance(ll_to_earth(lat, lon), ll_to_earth('45.539', '-122.387')) AS distance FROM business
    WHERE earth_box(ll_to_earth ('45.539', '-122.387'), 10000) @> ll_to_earth (lat, lon)
    ORDER BY distance DESC
    LIMIT 5
    """
)

[('Limit  (cost=2821.24..2821.25 rows=5 width=88) (actual time=29.574..29.576 rows=5 loops=1)',),
 ('  ->  Sort  (cost=2821.24..2821.27 rows=10 width=88) (actual time=29.573..29.574 rows=5 loops=1)',),
 ("        Sort Key: (sec_to_gc(cube_distance((ll_to_earth((lat)::double precision, (lon)::double precision))::cube, '(-2392907.1474526017, -3772510.0199894756, 4552273.121087229)'::cube))) DESC",),
 ('        Sort Method: top-N heapsort  Memory: 25kB',),
 ('        ->  Seq Scan on business  (cost=0.00..2821.07 rows=10 width=88) (actual time=0.167..29.548 rows=67 loops=1)',),
 ("              Filter: ('(-2402907.146428373, -3782510.018965247, 4542273.122111457),(-2382907.14847683, -3762510.021013704, 4562273.120063)'::cube @> (ll_to_earth((lat)::double precision, (lon)::double precision))::cube)",),
 ('              Rows Removed by Filter: 9933',),
 ('Planning Time: 1.100 ms',),
 ('Execution Time: 29.607 ms',)]

In [13]:
just_execute(
    """
    CREATE INDEX IF NOT EXISTS business_location_idx ON business USING GIST (ll_to_earth(lat, lon));
    ANALYZE business;
    """
)

'DONE'

In [14]:
explain_analyze(
    """
    SELECT *, earth_distance(ll_to_earth(lat, lon), ll_to_earth('45.539', '-122.387')) AS distance FROM business
    WHERE earth_box(ll_to_earth ('45.539', '-122.387'), 10000) @> ll_to_earth (lat, lon)
    ORDER BY distance DESC
    LIMIT 5
    """
)


[('Limit  (cost=44.15..44.17 rows=5 width=88) (actual time=0.490..0.490 rows=5 loops=1)',),
 ('  ->  Sort  (cost=44.15..44.18 rows=10 width=88) (actual time=0.489..0.489 rows=5 loops=1)',),
 ("        Sort Key: (sec_to_gc(cube_distance((ll_to_earth((lat)::double precision, (lon)::double precision))::cube, '(-2392907.1474526017, -3772510.0199894756, 4552273.121087229)'::cube))) DESC",),
 ('        Sort Method: top-N heapsort  Memory: 25kB',),
 ('        ->  Bitmap Heap Scan on business  (cost=4.23..43.99 rows=10 width=88) (actual time=0.149..0.469 rows=67 loops=1)',),
 ("              Recheck Cond: ('(-2402907.146428373, -3782510.018965247, 4542273.122111457),(-2382907.14847683, -3762510.021013704, 4562273.120063)'::cube @> (ll_to_earth((lat)::double precision, (lon)::double precision))::cube)",),
 ('              Heap Blocks: exact=54',),
 ('              ->  Bitmap Index Scan on business_location_idx  (cost=0.00..4.23 rows=10 width=0) (actual time=0.042..0.043 rows=67 loops=1)',),
 ("

## Querying 'customer' table

### Find customers banned from a business

In [15]:
explain_analyze(
    """
    SELECT * FROM customer WHERE banned_from @> '{8338}'
    """
)

[('Seq Scan on customer  (cost=0.00..496.00 rows=4 width=67) (actual time=0.032..2.409 rows=8 loops=1)',),
 ("  Filter: (banned_from @> '{8338}'::integer[])",),
 ('  Rows Removed by Filter: 19992',),
 ('Planning Time: 0.066 ms',),
 ('Execution Time: 2.425 ms',)]

In [16]:
just_execute(
    """
    CREATE INDEX IF NOT EXISTS customer_banned_from_idx ON customer USING GIN (banned_from);
    ANALYZE customer;
    """
)

'DONE'

In [17]:
explain_analyze(
    """
    SELECT * FROM customer WHERE banned_from @> '{8338}'
    """
)

[('Bitmap Heap Scan on customer  (cost=12.85..27.37 rows=4 width=67) (actual time=0.072..0.082 rows=8 loops=1)',),
 ("  Recheck Cond: (banned_from @> '{8338}'::integer[])",),
 ('  Heap Blocks: exact=8',),
 ('  ->  Bitmap Index Scan on customer_banned_from_idx  (cost=0.00..12.84 rows=4 width=0) (actual time=0.065..0.066 rows=8 loops=1)',),
 ("        Index Cond: (banned_from @> '{8338}'::integer[])",),
 ('Planning Time: 0.149 ms',),
 ('Execution Time: 0.148 ms',)]

## Querying 'feedback' table

### Get feedback for a specific business

In [18]:
explain_analyze(
    """
    SELECT * FROM feedback WHERE business_id = 2
    """
)

[('Seq Scan on feedback  (cost=0.00..3170.15 rows=9 width=187) (actual time=0.009..6.617 rows=9 loops=1)',),
 ('  Filter: (business_id = 2)',),
 ('  Rows Removed by Filter: 80003',),
 ('Planning Time: 0.061 ms',),
 ('Execution Time: 6.636 ms',)]

In [19]:
just_execute(
    """
    CREATE INDEX IF NOT EXISTS feedback_business_idx ON feedback USING BTREE (business_id);
    ANALYZE feedback;
    """
)

'DONE'

In [20]:
explain_analyze(
    """
    SELECT * FROM feedback WHERE business_id = 2
    """
)

[('Index Scan using feedback_business_idx on feedback  (cost=0.29..8.47 rows=9 width=188) (actual time=0.046..0.047 rows=9 loops=1)',),
 ('  Index Cond: (business_id = 2)',),
 ('Planning Time: 0.155 ms',),
 ('Execution Time: 0.058 ms',)]

### Get feedback for a specific customer

In [21]:
explain_analyze(
    """
    SELECT * 
    FROM feedback f INNER JOIN customer c ON f.customer_id = c.customer_id 
    WHERE c.name = 'Mr. Sean Hubbard 196'
    """
)

[('Hash Join  (cost=496.01..3676.21 rows=4 width=255) (actual time=3.418..10.156 rows=6 loops=1)',),
 ('  Hash Cond: (f.customer_id = c.customer_id)',),
 ('  ->  Seq Scan on feedback f  (cost=0.00..2970.12 rows=80012 width=188) (actual time=0.005..3.264 rows=80012 loops=1)',),
 ('  ->  Hash  (cost=496.00..496.00 rows=1 width=67) (actual time=1.794..1.795 rows=1 loops=1)',),
 ('        Buckets: 1024  Batches: 1  Memory Usage: 9kB',),
 ('        ->  Seq Scan on customer c  (cost=0.00..496.00 rows=1 width=67) (actual time=0.004..1.791 rows=1 loops=1)',),
 ("              Filter: (name = 'Mr. Sean Hubbard 196'::text)",),
 ('              Rows Removed by Filter: 19999',),
 ('Planning Time: 0.315 ms',),
 ('Execution Time: 10.178 ms',)]

In [22]:
just_execute(
    """
    CREATE INDEX IF NOT EXISTS customer_name_idx ON customer USING BTREE (name);
    ANALYZE customer;
    """
)

'DONE'

In [23]:
explain_analyze(
    """
    SELECT * 
    FROM feedback f INNER JOIN customer c ON f.customer_id = c.customer_id 
    WHERE c.name = 'Mr. Sean Hubbard 196'
    """
)

[('Hash Join  (cost=8.32..3188.51 rows=4 width=255) (actual time=0.454..7.223 rows=6 loops=1)',),
 ('  Hash Cond: (f.customer_id = c.customer_id)',),
 ('  ->  Seq Scan on feedback f  (cost=0.00..2970.12 rows=80012 width=188) (actual time=0.005..3.241 rows=80012 loops=1)',),
 ('  ->  Hash  (cost=8.30..8.30 rows=1 width=67) (actual time=0.019..0.021 rows=1 loops=1)',),
 ('        Buckets: 1024  Batches: 1  Memory Usage: 9kB',),
 ('        ->  Index Scan using customer_name_idx on customer c  (cost=0.29..8.30 rows=1 width=67) (actual time=0.017..0.018 rows=1 loops=1)',),
 ("              Index Cond: (name = 'Mr. Sean Hubbard 196'::text)",),
 ('Planning Time: 0.268 ms',),
 ('Execution Time: 7.248 ms',)]

In [24]:
just_execute(
    """
    CREATE INDEX IF NOT EXISTS feedback_customer_idx ON feedback USING BTREE (customer_id);
    ANALYZE feedback;
    """
)

'DONE'

In [25]:
explain_analyze(
    """
    SELECT * 
    FROM feedback f INNER JOIN customer c ON f.customer_id = c.customer_id 
    WHERE c.name = 'Mr. Sean Hubbard 196'
    """
)

[('Nested Loop  (cost=4.61..28.20 rows=4 width=255) (actual time=0.018..0.026 rows=6 loops=1)',),
 ('  ->  Index Scan using customer_name_idx on customer c  (cost=0.29..8.30 rows=1 width=67) (actual time=0.008..0.009 rows=1 loops=1)',),
 ("        Index Cond: (name = 'Mr. Sean Hubbard 196'::text)",),
 ('  ->  Bitmap Heap Scan on feedback f  (cost=4.32..19.86 rows=4 width=188) (actual time=0.008..0.014 rows=6 loops=1)',),
 ('        Recheck Cond: (customer_id = c.customer_id)',),
 ('        Heap Blocks: exact=6',),
 ('        ->  Bitmap Index Scan on feedback_customer_idx  (cost=0.00..4.32 rows=4 width=0) (actual time=0.003..0.003 rows=6 loops=1)',),
 ('              Index Cond: (customer_id = c.customer_id)',),
 ('Planning Time: 0.262 ms',),
 ('Execution Time: 0.042 ms',)]

### Find exact feedback

In [26]:
explain_analyze(
    """
    SELECT * FROM feedback 
    WHERE message = 'Lose situation develop let few increase staff. Point quality baby cultural head. Likely police reality guess upon yes successful suffer. Who center network threat. Then majority establish behavior popular. Direction throw arrive animal.'
    """
)

[('Seq Scan on feedback  (cost=0.00..3170.15 rows=1 width=188) (actual time=0.009..6.887 rows=1 loops=1)',),
 ("  Filter: (message = 'Lose situation develop let few increase staff. Point quality baby cultural head. Likely police reality guess upon yes successful suffer. Who center network threat. Then majority establish behavior popular. Direction throw arrive animal.'::text)",),
 ('  Rows Removed by Filter: 80011',),
 ('Planning Time: 0.056 ms',),
 ('Execution Time: 6.906 ms',)]

In [27]:
just_execute(
    """
    -- can't use BTREE due to 'index row size 3288 exceeds btree version 4 maximum 2704 for index "business_message_idx"'
    CREATE INDEX IF NOT EXISTS feedback_message_idx ON feedback USING HASH (message);
    ANALYZE feedback;
    """
)

'DONE'

In [28]:
explain_analyze(
    """
    SELECT * FROM feedback 
    WHERE message = 'Lose situation develop let few increase staff. Point quality baby cultural head. Likely police reality guess upon yes successful suffer. Who center network threat. Then majority establish behavior popular. Direction throw arrive animal.'
    """
)

[('Index Scan using feedback_message_idx on feedback  (cost=0.00..8.02 rows=1 width=188) (actual time=0.013..0.013 rows=1 loops=1)',),
 ("  Index Cond: (message = 'Lose situation develop let few increase staff. Point quality baby cultural head. Likely police reality guess upon yes successful suffer. Who center network threat. Then majority establish behavior popular. Direction throw arrive animal.'::text)",),
 ('Planning Time: 0.123 ms',),
 ('Execution Time: 0.023 ms',)]